<a href="https://colab.research.google.com/github/helderthadeu/ML-DIO-1-TransferLearning/blob/main/notebooks/transfer-learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Transfer Learning Project**

In [ ]:
import keras
import numpy as np

import random
import zipfile
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

In [ ]:
# Baixar o dataset
!wget --no-check-certificate \
    https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip \
    -O /tmp/catsdogs.zip

# Extrair os arquivos
zip_ref = zipfile.ZipFile('/tmp/catsdogs.zip', 'r')
zip_ref.extractall('/tmp/catsdogs')
zip_ref.close()

--2025-07-10 21:37:08--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 2.22.76.162, 2a02:26f0:6d00:3b6::317f, 2a02:26f0:6d00:39f::317f
Connecting to download.microsoft.com (download.microsoft.com)|2.22.76.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘/tmp/catsdogs.zip’

/tmp/catsdogs.zip   100%[===================>] 786.67M   174MB/s    in 5.0s    

2025-07-10 21:37:13 (157 MB/s) - ‘/tmp/catsdogs.zip’ saved [824887076/824887076]



In [ ]:
import os
from PIL import Image

# Caminho para as pastas de imagens
cat_dir = '/tmp/catsdogs/PetImages/Cat'
dog_dir = '/tmp/catsdogs/PetImages/Dog'

image_dirs = [cat_dir, dog_dir]

# Itera sobre as pastas de cães e gatos
for dir_path in image_dirs:
    for filename in os.listdir(dir_path):
        image_path = os.path.join(dir_path, filename)
        try:
            # Tenta abrir a imagem
            with Image.open(image_path) as img:
                img.verify()  # Verifica se é uma imagem válida
        except (IOError, SyntaxError, Image.UnidentifiedImageError) as e:
            # Se der erro, o arquivo está corrompido
            print(f'Arquivo corrompido encontrado e removido: {image_path}')
            os.remove(image_path)

Arquivo corrompido encontrado e removido: /tmp/catsdogs/PetImages/Cat/666.jpg
Arquivo corrompido encontrado e removido: /tmp/catsdogs/PetImages/Cat/Thumbs.db
Arquivo corrompido encontrado e removido: /tmp/catsdogs/PetImages/Dog/Thumbs.db
Arquivo corrompido encontrado e removido: /tmp/catsdogs/PetImages/Dog/11702.jpg


In [ ]:
# Configurações
IMG_SIZE = (224, 224)  # Tamanho esperado pela VGG
BATCH_SIZE = 32
NUM_CLASSES = 2

In [ ]:
# Criar geradores de dados
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% para validação
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Gerador para treino
train_generator = train_datagen.flow_from_directory(
    '/tmp/catsdogs/PetImages',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

# Gerador para validação
val_generator = train_datagen.flow_from_directory(
    '/tmp/catsdogs/PetImages',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 20000 images belonging to 2 classes.
Found 4998 images belonging to 2 classes.


In [ ]:
# Carregando o modelo VGG16 já treinado da base de dados do KErs
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)

Este código está modifica a arquitetura da rede neural VGG para substituir sua camada de classificação original por uma nova camada personalizada.

In [ ]:
#Cria uma referencia a camada de entrada da VGG
inp = vgg.input

#Cria uma nova camada de clasificação

new_classification_layer = Dense(2, activation='softmax')

#Conecta esta nova camada a penúltima da VGG
out = new_classification_layer(vgg.layers[-2].output)

#Cria um novo modelo, mantendo todas as camadas originais
model_new = Model(inp,out)

In [ ]:
# Garante que todos os nós estão com seus pesos congelados, "travando eles"
for l, layer in enumerate(model_new.layers):
    layer.trainable = False

# Garante que o último nó será treinado
for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True

model_new.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_new.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         8,194 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,268,738 (512.19 MB)

 Trainable params: 8,194 (32.01 KB)

 Non-trainable params: 134,260,544 (512.16 MB)

In [ ]:
history2 = model_new.fit(train_generator,
                        steps_per_epoch=train_generator.samples // BATCH_SIZE,
                         batch_size=32,
                         epochs=10,
                         validation_data=val_generator,
                         validation_steps=val_generator.samples // BATCH_SIZE
)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 363s 577ms/step - accuracy: 0.7598 - loss: 0.4880 - val_accuracy: 0.8484 - val_loss: 0.3420
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 374s 566ms/step - accuracy: 0.8539 - loss: 0.3267 - val_accuracy: 0.8694 - val_loss: 0.2941
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 381s 565ms/step - accuracy: 0.8622 - loss: 0.3105 - val_accuracy: 0.8694 - val_loss: 0.2955
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 388s 575ms/step - accuracy: 0.8645 - loss: 0.3117 - val_accuracy: 0.8782 - val_loss: 0.2855
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 368s 552ms/step - accuracy: 0.8686 - loss: 0.2955 - val_accuracy: 0.8638 - val_loss: 0.3129
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 383s 554ms/step - accuracy: 0.8748 - loss: 0.2887 - val_accuracy: 0.8421 - val_loss: 0.3613
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 348s 556ms/step - accuracy: 0.8747 - loss: 0.2922 - val_accuracy: 0.8738 - val_loss: 0.2921
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 380s 553ms/step - accuracy: 0.8734 -